In [1]:
test_input = """#.#####################
#.......#########...###
#######.#########.#.###
###.....#.>.>.###.#.###
###v#####.#v#.###.#.###
###.>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>.#
#.#.#v#######v###.###v#
#...#.>.#...>.>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>.>.#.>.###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################.#"""

In [2]:
input = open("inputs/23").read()

In [3]:
import numpy as np

In [4]:
def parse_board(input):
    return np.array(list(map(list, input.splitlines())))

def in_bounds(shape, pos):
    return all(0 <= pos[i] < shape[i] for i in range(len(shape)))

def add_tuples(tuple1, tuple2):
    return tuple(map(lambda x, y: x + y, tuple1, tuple2))

directions= {
    "right": (0, 1),
    "left": (0, -1),
    "up": (-1, 0),
    "down": (1, 0)
}

arrow_to_direction = {
    ">": "right",
    "<": "left",
    "v": "down",
    "^": "up"
}

directions_list = list(directions.values())
directions_list

[(0, 1), (0, -1), (-1, 0), (1, 0)]

In [5]:
# board = parse_board(test_input)
board = parse_board(input)

In [6]:
empty_pos = list(zip(*np.where(board == '.')))

empty_pos_set = set(empty_pos)
len(empty_pos)

9288

In [7]:
slope_pos = list(zip(*np.where((board != '.') & (board != "#"))))
slope_pos_set = set(slope_pos)
len(slope_pos)

118

In [8]:
N = board.shape[0]

start = (0, 1)
end = (N-1, N-2)

In [9]:
start, end

((0, 1), (140, 139))

In [10]:
import networkx as nx

In [11]:
g = nx.DiGraph()

for p in empty_pos_set | slope_pos_set:
    for dir in directions_list:
        n = add_tuples(p, dir)

        if (n in empty_pos_set) or (n in slope_pos_set):
            g.add_edge(p, n)

# for p in slope_pos_set:
#     forced_dir = directions[arrow_to_direction[board[p]]]
#     g.add_edge(p, add_tuples(p, forced_dir))

In [12]:
g.number_of_edges(), g.number_of_nodes()

(18860, 9406)

In [13]:
# from networkx.drawing.nx_agraph import to_agraph 


# A = to_agraph(g)
# A

# for node in A.nodes():
#     # node.attr['label'] = 'Custom Label'  # Set a custom label

#     module_type = node.attr['module_type']
#     node.attr['color'] = 'red' if module_type == '%' else 'blue' if module_type == '&' else 'black'

# for edge in A.edges():
#     edge.attr['label'] = edge.attr['index']

# A.layout()
# A.draw('23_graph_test.png', prog='dot')  # Save to file

In [14]:
# all_paths = list(nx.all_simple_paths(g, start, end))

In [15]:
# len(all_paths)

In [16]:
# max(len(l) for l in all_paths) - 1

In [17]:
# reprocess the graph such that long "tubes" get collapsed into a single node

In [18]:
tube_nodes = []

for v in g.nodes():
    if len(list(g.neighbors(v))) == 2:
        tube_nodes.append(v)
    else:
        print(f"Non tube node: {v}")

Non tube node: (31, 81)
Non tube node: (133, 81)
Non tube node: (77, 131)
Non tube node: (63, 125)
Non tube node: (31, 61)
Non tube node: (29, 103)
Non tube node: (11, 61)
Non tube node: (61, 15)
Non tube node: (103, 17)
Non tube node: (135, 129)
Non tube node: (109, 125)
Non tube node: (55, 107)
Non tube node: (101, 65)
Non tube node: (67, 89)
Non tube node: (89, 83)
Non tube node: (77, 57)
Non tube node: (89, 105)
Non tube node: (31, 9)
Non tube node: (77, 17)
Non tube node: (53, 61)
Non tube node: (0, 1)
Non tube node: (61, 41)
Non tube node: (11, 33)
Non tube node: (85, 35)
Non tube node: (107, 99)
Non tube node: (111, 77)
Non tube node: (41, 123)
Non tube node: (15, 17)
Non tube node: (107, 37)
Non tube node: (131, 31)
Non tube node: (31, 33)
Non tube node: (140, 139)
Non tube node: (11, 77)
Non tube node: (129, 101)
Non tube node: (13, 111)
Non tube node: (133, 65)


In [19]:
len(tube_nodes) / g.number_of_nodes()

0.9961726557516479

In [20]:
tube_subgraph = g.subgraph(tube_nodes)

In [21]:
len(tube_subgraph) / len(g)

0.9961726557516479

In [22]:
connected_tubes = list(nx.strongly_connected_components(tube_subgraph))

In [23]:
len(connected_tubes)

60

In [24]:
tube_to_connected_tube = {}

for i, tube in enumerate(connected_tubes):
    for node in tube:
        tube_to_connected_tube[node] = i

In [25]:
tube_g = nx.DiGraph()

In [26]:
for u, v in g.edges():
    if (u in tube_nodes) and (v in tube_nodes):
        # this implies they're in the same tube. I think.
        pass

    if (u in tube_nodes) and (v not in tube_nodes):
        tube_num = tube_to_connected_tube[u]

        tube_g.add_edge(f"tube_{tube_num}", v, weight=1)
    
    if (u not in tube_nodes) and (v in tube_nodes):
        tube_num = tube_to_connected_tube[v]

        tube_g.add_edge(u, f"tube_{tube_num}", weight=len(connected_tubes[tube_num]))

    if (u not in tube_nodes) and (v not in tube_nodes):
        print('hmm')
        tube_g.add_edge(u, v, weight=1)

In [27]:
tube_g.number_of_nodes(), tube_g.number_of_edges()

(96, 240)

In [28]:
from networkx.drawing.nx_agraph import to_agraph 


A = to_agraph(tube_g)
A

# for node in A.nodes():
#     # node.attr['label'] = 'Custom Label'  # Set a custom label

#     module_type = node.attr['module_type']
#     node.attr['color'] = 'red' if module_type == '%' else 'blue' if module_type == '&' else 'black'

for edge in A.edges():
    edge.attr['label'] = edge.attr['weight']

A.layout()
A.draw('23_graph_test_tube.png', prog='dot')  # Save to file

In [29]:
all_paths = list(nx.all_simple_paths(tube_g, start, end))

In [30]:
len(all_paths)

1262816

In [31]:
import math

max_w = -math.inf

for p in all_paths:
    w = 0

    for u, v in zip(p, p[1:]):
        w += tube_g.get_edge_data(u, v)['weight']

    # print(final_w)

    if w > max_w:
        max_w = w

max_w

6398

In [34]:
g.number_of_nodes(), g.number_of_edges()

(9406, 18860)